In [13]:
"""
Data Source: http://developer.nytimes.com/io-docs (NY Times API)
Objective: To return the top 20 most popular headlines on NY Times. Popularity is calculated on a daily basis.
"""

In [ ]:
import requests
import json

# Connect to the NYTimes API and fetch the data
def getNYTimesData():
    ''' 
    Connects to the API, and returns back a list
    of dictionaries, each dictionary corresponding
    to a different news article
    '''
    # Let's get the data from the NYTimes API
    url = 'http://api.nytimes.com/svc/mostpopular/v2/mostviewed/all-sections/1.json?api-key=b47e2f541e13e587fbae8e1e9ae41c8f%3A4%3A74553739'
    resp = requests.get(url)
    results = json.loads(resp.text)
    data = results["results"]
    return data

NYTimes_data = getNYTimesData()




In [14]:
import MySQLdb as mdb
import sys

# Setup the database in which we will store the NYTimes data
def connectDB():
    con = mdb.connect(host = 'localhost', 
                      user = 'root', 
                      passwd = 'dwdstudent2015', 
                      charset = 'utf8', use_unicode=True);
    return con

def createNYTimesDB(con, db_name):
    ''' 
    Connects to the database and creates (if it does not exist)
    the database and the tables needed to store the data
    '''
    # Query to create a database
    create_db_query = "CREATE DATABASE IF NOT EXISTS {0} DEFAULT CHARACTER SET 'utf8'".format(db_name)

    # Create a database
    cursor = con.cursor()
    cursor.execute(create_db_query)
    cursor.close()
    pass


def createTimeInvariantTable(con, db_name, table_name):
    cursor = con.cursor()
    # Create a table
    # The {0} and {1} are placeholders for the parameters in the format(....) statement
    create_table_query = '''CREATE TABLE IF NOT EXISTS {0}.{1} 
                                    (url varchar(250),
                                    title varchar(250),
                                    byline varchar(250),
                                    section varchar(250), 
                                    published_date datetime,
                                    abstract varchar(500),
                                    PRIMARY KEY(url)
                                    )'''.format(db_name, table_name)
    cursor.execute(create_table_query)
    cursor.close()

    
def createTimeVaryingTable(con, db_name, table_name):
    cursor = con.cursor()
    # Create a table
    # The {0} and {1} are placeholders for the parameters in the format(....) statement
    create_table_query = '''CREATE TABLE IF NOT EXISTS {0}.{1} 
                                    (url varchar(250),
                                    title varchar(250),
                                    views int,
                                    PRIMARY KEY(url)
                                    )'''.format(db_name, table_name)
    cursor.execute(create_table_query)
    cursor.close()

con = connectDB()
db_name = 'NYTimes'
createNYTimesDB(con, db_name)
group_table = 'headlines_info'
createTimeInvariantTable(con, db_name, group_table)
group_table = 'headlines_rank'
createTimeVaryingTable(con, db_name, group_table)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: Warning: Can't create database 'NYTimes'; database exists
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: Warning: Table 'headlines_info' already exists
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:54: Warning: Table 'headlines_rank' already exists


In [15]:
from datetime import datetime

def storeTimeInvariantData(con, NYTimes_data):
    '''
    Accepts as a parameter a list of dictionaries, where
    each dictionary is a NYTimes article.
    Goes over these dictionaries, and stores in the database
    the entries that are time invariant.
    We need to check if the article already exists,
    and if it does not, store it in the database
    '''
    db_name = 'NYTimes'
    table_name = 'headlines_info'
    
    for headline in NYTimes_data:
        url = headline["url"]
        title = headline["title"]
        author = headline["byline"]
        topic = headline["section"]
        date_str = headline["published_date"]
        date = datetime.strptime(date_str, '%Y-%m-%d')
        abstract = headline["abstract"]
        insertHeadline(con, db_name, table_name, 
                      url, title, author, topic, date, abstract)
    
    # Writes the data in the database, for sure
    con.commit()
    return
 
def insertHeadline(con, db_name, table_name, 
                  url, title, author, topic, date, abstract):
    query_template = '''INSERT IGNORE INTO {0}.{1}(url, 
                                    title, 
                                    byline,
                                    section,
                                    published_date,
                                    abstract) 
                VALUES (%s, %s, %s, %s, %s, %s)'''.format(db_name, table_name)

    cursor = con.cursor()
    query_parameters = (url, title, author, topic, date, abstract)
    cursor.execute(query_template, query_parameters)
    cursor.close()


storeTimeInvariantData(con, NYTimes_data)

In [16]:
# Go over the NYTimes data and store the time-varying
# data into the appropriate table in the database
def storeTimeVaryingData(con, NYTimes_data):
    '''
    Accepts as a parameter a list of dictionaries, where
    each dictionary is a NYTimes article.
    Goes over these dictionaries, and stores in the database
    the entries that are time invariant.
    We need to check if the article (using URL as primary key) already exists in the ranks,
    and if it does not, store it in the database
    '''
    db_name = 'NYTimes'
    table_name = 'headlines_rank'
    
    for headline in NYTimes_data:
        url = headline["url"]
        title = headline["title"]
        rank = headline["views"]
        insertRank(con, db_name, table_name, 
            url, title, rank)

    con.commit()
    
    return

def insertRank(con, db_name, table_name, 
                  url, title, rank):
    query_template = '''INSERT INTO {0}.{1}(url, 
                                    title,
                                    views) 
                VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE views=VALUES(views)'''.format(db_name, table_name)

    cursor = con.cursor()
    query_parameters = (url, title, rank)
    cursor.execute(query_template, query_parameters)
    cursor.close()

    return

def deleteTimeVaryingData(con):
    
    db_name = 'NYTimes'
    table_name = 'headlines_rank'
    
    query_template = '''DELETE FROM {0}.{1}'''.format(db_name, table_name)
    cursor = con.cursor()
    cursor.execute(query_template)
    cursor.close()

    return

deleteTimeVaryingData(con)
storeTimeVaryingData(con, NYTimes_data)